In [94]:
# Initial Python environment setup...
import googlemaps
import simplejson, urllib.request
import numpy as np # linear algebra
import pandas as pd # CSV file I/O (e.g. pd.read_csv)
import os # reading the input files we have access to
import json
import requests
import time
from tqdm._tqdm_notebook import tqdm_notebook as tqdm


print(os.listdir('input/'))

['sample_submission.csv', 'test.csv', 'train.csv', 'train.csv.zip']


In [109]:
tqdm.pandas()
train_df =  pd.read_csv('input/train.csv', nrows = 1000)
train_df.dtypes

key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [112]:
# remove nan
print(train_df.isnull().sum())
print('Old size: %d' % len(train_df))
train_df = train_df.dropna(how = 'any', axis = 'rows')
print('New size: %d' % len(train_df))
#plot = train_df.iloc[:2000].plot.scatter('abs_diff_longitude', 'abs_diff_latitude')
#pri#t('Old size: %d' % len(train_df))
#train_df = train_df[(train_df.abs_diff_longitude < 5.0) & (train_df.abs_diff_latitude < 5.0)]
#print('New size: %d' % len(train_df))


key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64
Old size: 1000
New size: 1000


In [117]:
def google_api_find_duration_distance_driving(origins,destinations,departure_time):
    
    orig_lat,orig_lng = origins
    dest_lat, dest_lng = destinations
    departure_time='2019'+departure_time[4:]
    pattern = '%Y-%m-%d %H:%M:%S'
    departure_time = int(time.mktime(time.strptime(departure_time, pattern)))
    orig_coord = orig_lat, orig_lng
    dest_coord = dest_lat, dest_lng
    params={
        "units":'metric',
        "key":API_KEY,
        "departure_time":departure_time,
        "origins":str(orig_lat)+','+str(orig_lng),
        "destinations":str(dest_lat)+','+str(dest_lng)
    }
    r = requests.post('https://maps.googleapis.com/maps/api/distancematrix/json?transit_mode=driving&departure_time={departure_time}&origins={origins}&destinations={destinations}'.
                      format(key=params['key'],units=params['units'],departure_time=params['departure_time'],origins=params['origins'],
                            destinations=params['destinations']))
    response_json= r.json()
    if response_json['status']=='OK':
        if response_json['rows'][0]['elements'][0]['status']=='OK':
            return response_json['rows'][0]['elements'][0]['duration']['value'],response_json['rows'][0]['elements'][0]['distance']['value']
    print(response_json)
    return None,None

def OSM_find_duration_distance_driving(origins,destinations):
    orig_lat,orig_lng = origins
    dest_lat, dest_lng = destinations
    orig_coord = orig_lat, orig_lng
    dest_coord = dest_lat, dest_lng
    params={
        "origins":str(orig_lng)+','+str(orig_lat),
        "destinations":str(dest_lng)+','+str(dest_lat)
    }
    to_send = params['origins']+';'+params['destinations']
    primary_url_test_server = "http://router.project-osrm.org/route/v1/driving/"
    primary_url_local_server="http://127.0.0.1:5000/route/v1/driving/"
    url = primary_url_test_server+to_send
    response = requests.get(url)
    data = json.loads(response.content)
    
    if response.status_code == 200:
        return data['routes'][0]['duration'],data['routes'][0]['distance']
    else:
        print(url)
        print(data)
        return None,None


def create_duration_distance_columns(df):
    origins = (df['pickup_latitude'],df['pickup_longitude'])
    destinations =(df['dropoff_latitude'],df['dropoff_longitude'])
    departure_time = df['pickup_datetime'][:-4]
    #duration,distance=google_api_find_duration_distance_driving(origins,destinations,departure_time)
    duration,distance=OSM_find_duration_distance_driving(origins,destinations)
    df['duration']= duration
    df['distance']= distance
    return df
train_df= train_df.progress_apply(create_duration_distance_columns, axis=1)

http://router.project-osrm.org/route/v1/driving/-73.844311,40.721319;-73.84161,40.712278000000005
{'message': 'Too Many Requests'}
http://router.project-osrm.org/route/v1/driving/-73.844311,40.721319;-73.84161,40.712278000000005
{'message': 'Too Many Requests'}
http://router.project-osrm.org/route/v1/driving/-74.016048,40.711303;-73.979268,40.782004
{'message': 'Too Many Requests'}
http://router.project-osrm.org/route/v1/driving/-73.982738,40.76127;-73.991242,40.750562
{'message': 'Too Many Requests'}
http://router.project-osrm.org/route/v1/driving/-73.98713000000001,40.733143;-73.991567,40.758092
{'message': 'Too Many Requests'}
http://router.project-osrm.org/route/v1/driving/-73.96809499999999,40.768008;-73.956655,40.783762
{'message': 'Too Many Requests'}
http://router.project-osrm.org/route/v1/driving/-74.000964,40.731629999999996;-73.972892,40.758233000000004
{'message': 'Too Many Requests'}
http://router.project-osrm.org/route/v1/driving/-73.980002,40.751661999999996;-73.973802,4

KeyboardInterrupt: 

In [106]:
# round lat/long
train_df['pickup_longitude_rounded']=round(train_df['pickup_longitude'],2)
train_df['pickup_latitude_rounded']=round(train_df['pickup_latitude'],2)
# round lat/long
train_df['dropoff_longitude_rounded']=round(train_df['dropoff_longitude'],2)
train_df['dropoff_latitude_rounded']=round(train_df['dropoff_latitude'],2)
#convert all date to echo datetime
def convert_datetime_to_echo(df):
    pattern = '%Y-%m-%d %H:%M:%S'
    df['pickup_datetime_echo'] = int(time.mktime(time.strptime(df['departure_time'], pattern)))
    return df
train_df= train_df.progress_apply(convert_datetime_to_echo, axis=1)
train_df.head(1)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_longitude_rounded,pickup_latitude_rounded,dropoff_longitude_rounded,dropoff_latitude_rounded
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.84161,40.712278,1,-73.8,40.7,-73.8,40.7


In [ ]:
from sklearn import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

feature_columns_to_use = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count',
                          'pickup_longitude_rounded','pickup_latitude_rounded',
                          'dropoff_longitude_rounded','dropoff_latitude_rounded']
X = train_df[feature_columns_to_use]
y=train_df['fare_amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train = X_train.as_matrix()
X_test = X_test.as_matrix()
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train, y_train)

y_predicted = gbm.predict(X_test)

rms = sqrt(mean_squared_error(y_test, y_predicted))


In [ ]:
to_predict_df = pd.read_csv('input/test.csv')
# Reuse the above helper functions to add our features and generate the input matrix.
add_travel_vector_features(to_predict_df)
to_predict_X = to_predict_df.as_matrix()

submission = pd.DataFrame(
    {'key': test_df.key, 'fare_amount': test_y_predictions},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission.csv', index = False)